In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [2]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/mlm', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/modules']


2024-06-04 03:07:59.604368: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-04 03:07:59.637538: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 03:08:00.386178: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from datasets import load_from_disk,concatenate_datasets


adapter_name="mlm_union_F"
dataset = load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/datasets")



In [4]:
reload(config)

    
filtered_target = dataset['train'].filter(lambda example: example['genre'] == "fiction").shuffle(seed=42)
#filtered_target = shuffled_filtered_target.train_test_split(test_size=0.1)


filtered_test_target = dataset['validation_matched'].filter(lambda example: example['genre'] == "fiction")
train_target = filtered_target
test_target = filtered_test_target

 

Filter:   0%|          | 0/392702 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9815 [00:00<?, ? examples/s]

In [5]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
init(model)
reload(fn)
fn.print_trainable_parameters(model)

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

trainable params: 109514298 || all params: 109514298 || trainable%: 100.0


In [6]:
from adapters import ConfigUnion, LoRAConfig, PrefixTuningConfig, SeqBnConfig,SeqBnInvConfig,AdapterConfig,LoRAConfig

union_config = ConfigUnion(
    SeqBnInvConfig(reduction_factor=2),
    LoRAConfig(r=8, alpha=16),
)
model.add_adapter(adapter_name, config=union_config)

model.train_adapter([adapter_name])
model.active_adapters = adapter_name
print(model.adapter_summary())
fn.print_trainable_parameters(model)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mlm_union_F              union             7,682,688       7.055       1       1
--------------------------------------------------------------------------------
Full model                               108,891,648     100.000               0
trainable params: 8274816 || all params: 117196986 || trainable%: 7.060604783812444


In [7]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(train_target,tokenizer)
tokenized_dataset

Map:   0%|          | 0/77348 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 77348
})

In [8]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[444:470]

results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [9]:
# we group texts and chunk them
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Map:   0%|          | 0/77348 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 17714
})

In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] have you any [MASK] to believe that [MASK] mistress was likely to take a sleeping powder [MASK] night [MASK] " [SEP] your mistress was [MASK] all night. [SEP] [CLS] do you want to see them? " [SEP] you can't [MASK] them. [SEP] [CLS] [MASK] groups your leader [MASK],'white said. [SEP] white said that was wherever your leader is. [SEP] [CLS] in the meantime [MASK] the young adventurers were sitting bolt upright, very events and ill at ease, in a taxi which, with [MASK] singular lack of originality commanded was [MASK] returning to the [MASK]tz via regent's park. [SEP] the adventurers walked home instead of taking [MASK] cab'

'>>> . [SEP] [CLS] my brother lawrence is convinced that we [MASK] making a fuss over nothing. [SEP] [MASK] brother is wrong when he thinks we are making a fuss [MASK] [SEP] [CLS] at eleven o'clock [MASK] tuppence made up her mind [MASK] [SEP] tuppence made a decision at [MASK] o'clock. [SEP] [CLS] a moment later he uttered a cry. [SEP] ten years later, 

In [11]:

downsampled_dataset = lm_datasets.train_test_split(
    test_size=0.1, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 15942
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1772
    })
})

In [12]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['test'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [13]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/56 [00:00<?, ?it/s]

>>> Perplexity: 14.24


In [14]:
trainer.train()

  0%|          | 0/9980 [00:00<?, ?it/s]

{'loss': 2.2011, 'grad_norm': 1.9283149242401123, 'learning_rate': 9.960000000000001e-05, 'epoch': 1.0}


  0%|          | 0/56 [00:00<?, ?it/s]

{'eval_loss': 1.8401819467544556, 'eval_runtime': 3.0766, 'eval_samples_per_second': 575.957, 'eval_steps_per_second': 18.202, 'epoch': 1.0}
{'loss': 1.9027, 'grad_norm': 1.5654383897781372, 'learning_rate': 9.476793248945148e-05, 'epoch': 2.0}


  0%|          | 0/56 [00:00<?, ?it/s]

...

  0%|          | 0/56 [00:00<?, ?it/s]

{'eval_loss': 1.5142015218734741, 'eval_runtime': 7.9118, 'eval_samples_per_second': 223.968, 'eval_steps_per_second': 7.078, 'epoch': 18.0}


Overwriting existing adapter 'mlm_union_F'.


{'train_runtime': 2307.1725, 'train_samples_per_second': 138.195, 'train_steps_per_second': 4.326, 'train_loss': 1.6774858522723073, 'epoch': 18.0}


TrainOutput(global_step=8982, training_loss=1.6774858522723073, metrics={'train_runtime': 2307.1725, 'train_samples_per_second': 138.195, 'train_steps_per_second': 4.326, 'train_loss': 1.6774858522723073, 'epoch': 18.0})

In [15]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/56 [00:00<?, ?it/s]

>>> Perplexity: 4.62


In [16]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)


: 

#### This is a report of mlm for FICTION domain using whole genre as target and then splitt after tokenization with seed 42. I used computer metrix for preplexity and early stopping 

The results before:
>>> Perplexity: 14.24

The results after:
>>> Perplexity: 4.62